In [10]:
import pandas as pd
import numpy as np
from collections import Counter
import ast
from sklearn.metrics import mean_squared_error

# Configuration for all functions
config = {
    "WS": {"min": 1.58, "max": 8.61, "lower": 3.07, "higher": 6.17},
    "PR": {"min": 2.07, "max": 10.0, "lower": 3.66, "higher": 6.11},
    "NR": {"min": 4.10, "max": 10.0, "lower": 2.06, "higher": 4.42},
    "SR": {"min": 2.29, "max": 10.0, "lower": 3.02, "higher": 6.67},
    "SFST": {"min": 0.0, "max": 7.71, "lower": 1.05, "higher": 6.51},
    "WS_Benefit": {"min": 0.08, "max": 10.0, "lower": 2.65, "higher": 6.50},
    "PR_Benefit": {"min": 0.49, "max": 10.0, "lower": 3.29, "higher": 6.68},
    "NR_Benefit": {"min": 0.71, "max": 10.0, "lower": 4.10, "higher": 7.76},
    "SR_Benefit": {"min": 0.49, "max": 8.79, "lower": 2.94, "higher": 6.19},
    "SFST_Benefit": {"min": 0.0, "max": 7.19, "lower": 1.86, "higher": 5.30}
}

# Normalize function
def normalize(values, min_val, max_val):
    return 10 * ((values - min_val) / (max_val - min_val))

# Classify function
def classify(value, lower, higher):
    if value < lower:
        return "lower"
    elif value <= higher:
        return "moderate"
    else:
        return "higher"

# Calculate accuracy function
def calculate_accuracy(actual, predicted):
    actual_classes = actual.split(',')
    predicted_classes = predicted.split(',')
    return sum(1 for a, p in zip(actual_classes, predicted_classes) if a == p) / len(actual_classes)

# Voting system for ensemble learning
def voting_classification(predictions):
    return [Counter(pred).most_common(1)[0][0] for pred in zip(*predictions)]

# Process each function defined in config
for function, params in config.items():
    for norm_type in ['non_norm', 'norm', 'non_norm_post']:
        test_csv_file = f"test_results_{function}_{norm_type}.csv"
        data = pd.read_csv(test_csv_file)

        # Filter out results with more than 5 features
        data = data[data['Number of Features'] <= 4]

        # Normalize and classify using the parameters specific to each function
        data['Normalized_Actual'] = data['Actual']
        data['Normalized_Predicted'] = data['Predicted']
            
        data['Classified_Actual'] = data['Normalized_Actual'].apply(lambda x: ','.join([classify(float(val), params['lower'], params['higher']) for val in x.split(',')]))
        data['Classified_Predicted'] = data['Normalized_Predicted'].apply(lambda x: ','.join([classify(float(val), params['lower'], params['higher']) for val in x.split(',')]))

        # Identify the top model based on lowest MSE
        top_model = data.nsmallest(1, 'MSE').iloc[0]

        # Extract features of the top model
        try:
            top_features = ast.literal_eval(top_model['Selected Features'])
        except (ValueError, SyntaxError):
            top_features = top_model['Selected Features'].split(', ')

        # Calculate overall accuracy for the top model
        top_model_accuracy = calculate_accuracy(top_model['Classified_Actual'], top_model['Classified_Predicted'])

        # Ensemble accuracy calculation
        top_5_models = data.nsmallest(5, 'MSE')
        ensemble_predictions = top_5_models['Classified_Predicted'].apply(lambda x: x.split(','))
        ensemble_predictions = np.array(ensemble_predictions.tolist())
        ensemble_actual = top_5_models.iloc[0]['Classified_Actual'].split(',')
        
        ensemble_votes = voting_classification(ensemble_predictions)
        ensemble_accuracy = calculate_accuracy(','.join(ensemble_actual), ','.join(ensemble_votes))

        # Calculate the MSE for the ensemble
        ensemble_actual_values = np.array([float(x) for x in top_5_models.iloc[0]['Normalized_Actual'].split(',')])
        ensemble_predicted_values = np.mean([np.array([float(x) for x in pred.split(',')]) for pred in top_5_models['Normalized_Predicted']], axis=0)
        ensemble_mse = mean_squared_error(ensemble_actual_values, ensemble_predicted_values)

        # Per-category accuracies for top model and ensemble
        categories = ["lower", "moderate", "higher"]
        top_model_category_accuracy = {category: 0 for category in categories}
        ensemble_category_accuracy = {category: 0 for category in categories}

        for category in categories:
            # Top model category accuracy
            top_model_category_count = top_model['Classified_Actual'].split(',').count(category)
            if top_model_category_count > 0:
                top_model_category_accuracy[category] = sum(1 for a, p in zip(top_model['Classified_Actual'].split(','), top_model['Classified_Predicted'].split(',')) if a == p == category) / top_model_category_count
            else:
                top_model_category_accuracy[category] = None
            
            # Ensemble category accuracy
            ensemble_category_count = ensemble_actual.count(category)
            if ensemble_category_count > 0:
                ensemble_category_accuracy[category] = sum(1 for a, p in zip(ensemble_actual, ensemble_votes) if a == p == category) / ensemble_category_count
            else:
                ensemble_category_accuracy[category] = None

        # Correct ensemble features concatenation
        ensemble_features = set()
        for _, row in top_5_models.iterrows():
            try:
                features = ast.literal_eval(row['Selected Features'])
            except (ValueError, SyntaxError):
                features = row['Selected Features'].split(', ')
            ensemble_features.update(features)

        # Print results
        print(f"\n{function} ({norm_type}) Results:")
        print(f"Top Model MSE: {top_model['MSE']:.4f}")
        print(f"Features used in Top Model: {', '.join(sorted(top_features))}")
        print(f"Overall Accuracy of Top Model: {top_model_accuracy:.2%}")
        for category in categories:
            top_acc = top_model_category_accuracy[category]
            if top_acc is not None:
                print(f"Top Model Accuracy for '{category}': {top_acc:.2%}")
            else:
                print(f"Top Model Accuracy for '{category}': N/A")

        print(f"\nEnsemble MSE: {ensemble_mse:.4f}")
        print(f"Ensemble Accuracy: {ensemble_accuracy:.2%}, Ensemble Features: {', '.join(sorted(ensemble_features))}")
        for category in categories:
            ensemble_acc = ensemble_category_accuracy[category]
            if ensemble_acc is not None:
                print(f"Ensemble Accuracy for '{category}': {ensemble_acc:.2%}")
            else:
                print(f"Ensemble Accuracy for '{category}': N/A")



WS (non_norm) Results:
Top Model MSE: 0.2627
Features used in Top Model: F22, F24, F31, F43
Overall Accuracy of Top Model: 80.95%
Top Model Accuracy for 'lower': 85.71%
Top Model Accuracy for 'moderate': 71.43%
Top Model Accuracy for 'higher': 100.00%

Ensemble MSE: 0.3738
Ensemble Accuracy: 73.81%, Ensemble Features: F22, F24, F30, F31, F43, F44, F46
Ensemble Accuracy for 'lower': 100.00%
Ensemble Accuracy for 'moderate': 47.62%
Ensemble Accuracy for 'higher': 100.00%

WS (norm) Results:
Top Model MSE: 0.4642
Features used in Top Model: F22, F31, F43
Overall Accuracy of Top Model: 90.48%
Top Model Accuracy for 'lower': 100.00%
Top Model Accuracy for 'moderate': 71.43%
Top Model Accuracy for 'higher': 100.00%

Ensemble MSE: 0.6676
Ensemble Accuracy: 90.48%, Ensemble Features: F22, F24, F30, F31, F43, F44, F46
Ensemble Accuracy for 'lower': 100.00%
Ensemble Accuracy for 'moderate': 71.43%
Ensemble Accuracy for 'higher': 100.00%

WS (non_norm_post) Results:
Top Model MSE: 0.5250
Feature